# Exercice - Nettoyage et pré-traitement de données

Source des données : [Demandes de valeurs foncières (DVF) géolocalisées](https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/)

Importer les bibliothèques Python nécessaires, telles que `pandas` pour manipuler les données :

In [ ]:
import pandas

from pandas.plotting import scatter_matrix

Extraire le jeu de données CSV avec la fonction `pandas.read_csv` :

In [ ]:
df = pandas.read_csv("https://files.data.gouv.fr/geo-dvf/latest/csv/2018/departements/38.csv.gz")
df

Analyser le jeu de données dans son ensemble (colonnes, lignes, types de données, valeurs manquantes) :

In [ ]:
df.info()

**Remarque** : un certain nombre de colonnes contiennent des valeurs nulles / manquantes ! En fonction des objectifs, certaines ne nous sont pas nécessairement utiles (`adresse_numero`), d'autres sont primordiales (`valeur_fonciere`, `surface_reelle_bati`, `nombre_pieces_principales`). Gardons cela en tête pour la suite, il est très probable que nous aillons à effectuer un nettoyage de données.

Afficher les statistiques des variables quantitatives :

In [ ]:
df.describe()

Sélectionner uniquement les colonnes d'intérêt :

In [ ]:
raw_dataset = df.loc[:, ["date_mutation", "nature_mutation", "valeur_fonciere", "nom_commune", "type_local", "surface_reelle_bati", "nombre_pieces_principales", "latitude", "longitude"]]
raw_dataset

Filtrer uniquement les ventes d'appartements :

In [ ]:
filtered_dataset = raw_dataset[(raw_dataset["nature_mutation"] == "Vente") & ((raw_dataset["type_local"] == "Appartement"))]
filtered_dataset

In [ ]:
filtered_dataset.info()

Certaines ventes ont des informations manquantes : valeur foncière, surface réelle, nombre de pièces principales, latitude, longitude.

Observons ces données incomplètes. Ignorons pour le moment la latitude et longitude car cela représente un échantillon conséquent :

In [ ]:
filtered_dataset[filtered_dataset[["valeur_fonciere", "surface_reelle_bati", "nombre_pieces_principales"]].isna().any(axis=1)]

À ce stade il nous est impossible de trouver des valeurs de remplacement approximées (note : nous pourrions utiliser des méthodes de regroupement statistique) donc allons supprimer les lignes ayant des valeurs manquantes pour les caractéristiques valeur foncière, surface réelle et nombre de pièces principales :

In [ ]:
clean_dataset = filtered_dataset[filtered_dataset[["valeur_fonciere", "surface_reelle_bati", "nombre_pieces_principales"]].notna().all(axis=1)]
clean_dataset.info()

Terminons avec les latitude et longitude manquantes. Nous pouvons approximer ces données manquantes de plusieurs manières :

- Effectuer un rapprochement avec d'autres caractéristiques mises de côté initialement (nom de rue, code postal, etc.)
- Appliquer une constante, par exemple les coordonnées du centre de la ville de Grenoble
- Appliquer les valeurs moyennes ou médianes de notre jeu de données

Nous allons utiliser la valeur médiane pour ces valeurs manquante, car c'est un mesure statistique dite "robuste", et cela permettra de ne pas changer la distribution statistique du jeu de données :

In [ ]:
lat_lng_median = clean_dataset[["latitude", "longitude"]].median()
lat_lng_median

In [ ]:
clean_dataset.loc[:, ["latitude", "longitude"]] = clean_dataset[["latitude", "longitude"]].fillna(lat_lng_median)
clean_dataset.info()

Maintenant que nous avons rapidement pré-traité nos données, explorons la distribution et la correlation statistique entre certaines caractéristiques :

In [ ]:
_ = scatter_matrix(clean_dataset[["valeur_fonciere", "surface_reelle_bati", "nombre_pieces_principales"]], figsize=(12, 8))

**Questions** :

- Que pouvons-nous observer ?
- Que pourrions effectuer comme analyses et traitements supplémentaires ?